In [9]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to
[nltk_data]     /home/jjordanoc/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [11]:
from nltk.corpus import cmudict
d = cmudict.dict()

In [2]:
from faster_whisper import WhisperModel

In [13]:
import librosa
import numpy as np
import glob
import os
import regex as re

In [3]:
# Run on GPU with FP16
model = WhisperModel("large-v2", device="cuda", compute_type="float16", download_root="faster-whisper/")

In [1]:
a, b = model.transcribe("audio-data/001 - Low/Avalinguo - Xoca and Josué segment 80 - J.mp3")

NameError: name 'model' is not defined

In [ ]:
a, b = model.transcribe("C:/Users/rojot/OneDrive/Escritorio/machine learning/repo/project/audio-data/001 - Low/Avalinguo - Xoca and Josué segment 80 - J.mp3")

In [15]:
def syllables_aux(word: str) -> int:
    #referred from stackoverflow.com/questions/14541303/count-the-number-of-syllables-in-a-word
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if(len(word)>0):
        if word[0] in vowels:
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:
                count +=1
        if word.endswith('e'):
            count -= 1
        if word.endswith('le'):
            count += 1
        if count == 0:
            count += 1
    return count

In [8]:
def count_word_syllables(word: str):
    try:
        # return the pronunciation with the minimum number of syllables
        return min([len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]])
    except KeyError:
        return syllables_aux(word)

In [9]:
def get_audio_duration(file_path: str):
   audio_data, sample_rate = librosa.load(file_path)
   duration = librosa.get_duration(y=audio_data, sr=sample_rate)
   return duration

In [10]:
def clean_word(word: str) -> str:
    lower = word.lower()
    new_str = ""
    for c in lower:
        if c.isalpha():
            new_str += c
    return new_str

In [11]:
def is_filler_word(word: str) -> str:
    return re.fullmatch(r"u+m+|e+m+|e+r+|e+h+|u+h+", word) is not None

In [21]:
def features_yu(audio_file: str) -> np.array:
    """
    (1) articulation rate = number of syllables, including disfluencies, divided by to-
    tal duration of speech apart from all (silent and filled) pauses longer than
    0.25 seconds;
    (2) speech rate = as for articulation rate, but including all pauses in the total
    speech duration;
    (3) effective speech rate = as for speech rate, but excluding disfluencies from the
    syllable count;
    (4) number of silent pauses above 0.25 seconds in duration;
    (5) mean length of silent pauses longer than 0. 25 seconds;
    (6) number of filled pauses (uh, er, mm, etc.);
    (7) mean length of all filled pauses;
    (8) number of pauses = sum of (4) and (6);
    (9) mean length of pauses = mean of (5) and (7), weighted by their respective
    frequencies (items 4 and 6);
    (10) number of other disfluencies (repetitions, restarts, false starts, corrections);
    (11) mean length of fluent runs = mean number of syllables produced between
    silent pauses longer than 0.25 seconds;
    (12) phonation/time ratio, calculated on the basis of items 4 and 6, as a percentage
    of overall speech time = (total duration of speech without pauses, divided by
    total duration of speech including pauses) × 100
    """
    print("Heree")
    results, _ = model.transcribe(audio_file,
                              language="en", word_timestamps=True, beam_size=5)
    # audio_duration = get_audio_duration(audio_file)
    silent_pauses = []
    filled_pauses = []
    duration_of_speech_excluding_pauses = 0
    fluent_runs_syllables = []
    # n_segments = 0
    last_segment = None
    for segment in results:
        # n_segments += 1
        # assert n_segments <= 1
        print("New segment")
        if last_segment is not None:
            silent_pause = segment.words[0].start - last_segment.words[-1].end
            if silent_pause > 0.25:
                silent_pauses.append(silent_pause)
        for i in range(len(segment.words)):
            word = segment.words[i]
            print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))
            # Compute silent pauses
            if i <= len(segment.words) - 2:
                next = segment.words[i+1]
                silent_pause = next.start - word.end
                if silent_pause > 0.25:
                    silent_pauses.append(silent_pause)
            clean = clean_word(word.word)
            # Compute filled pauses
            if is_filler_word(clean):
                filled_pauses.append(word.end - word.start)
            else:
                fluent_runs_syllables.append(count_word_syllables(clean))
                duration_of_speech_excluding_pauses += word.end - word.start
        last_segment = segment

    articulation_rate = (sum(fluent_runs_syllables) + len(filled_pauses)) / duration_of_speech_excluding_pauses if duration_of_speech_excluding_pauses else 0
    speech_rate = (sum(fluent_runs_syllables) + len(filled_pauses)) / (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses))
    effective_speech_rate = sum(fluent_runs_syllables) / duration_of_speech_excluding_pauses if duration_of_speech_excluding_pauses else 0
    number_of_silent_pauses = len(silent_pauses)
    mean_length_of_silent_pauses = sum(silent_pauses) / number_of_silent_pauses if number_of_silent_pauses else 0
    number_of_filled_pauses = len(filled_pauses)
    mean_length_of_filled_pauses = sum(filled_pauses) / number_of_filled_pauses if number_of_filled_pauses else 0
    number_of_pauses = number_of_silent_pauses + number_of_filled_pauses
    mean_length_of_pauses = (mean_length_of_silent_pauses * number_of_silent_pauses + mean_length_of_filled_pauses * number_of_filled_pauses) / number_of_pauses if number_of_pauses else 0
    number_of_other_disfluencies = 0 # TODO
    mean_length_of_fluent_runs = sum(fluent_runs_syllables) / len(fluent_runs_syllables) if len(fluent_runs_syllables) else 0
    phonation_ratio = duration_of_speech_excluding_pauses / (duration_of_speech_excluding_pauses + sum(filled_pauses) + sum(silent_pauses))
    return np.array([
        articulation_rate,
        speech_rate,
        effective_speech_rate,
        number_of_silent_pauses,
        mean_length_of_silent_pauses,
        number_of_filled_pauses,
        mean_length_of_filled_pauses,
        number_of_pauses,
        mean_length_of_pauses,
        number_of_other_disfluencies,
        mean_length_of_fluent_runs,
        phonation_ratio
    ])

In [1]:
features_yu("audio-data/001 - Low/Avalinguo - Xoca and Josué segment 80 - J.mp3")

NameError: name 'features_yu' is not defined

# Extract features and save to file

In [21]:
PATH = "audio-data/"

In [22]:
N_FEATURES_YU = 12

In [23]:
audio_subdirectories = os.listdir(PATH)
audio_subdirectories.sort()
print('Audio Subdirs: ', audio_subdirectories)

Audio Subdirs:  ['001 - Low', '002 - Intermediate', '003 - High']


In [ ]:
def parse_audio_files(parent_dir, sub_dirs, file_ext='*.mp3'): # Audio Format
    features = np.empty((0, N_FEATURES_YU))
    labels = np.empty(0, dtype=np.int)
    for label, sub_dir in enumerate(sub_dirs):
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            print("Actual File Name: ", file_name)
            try:
                extracted_features = features_yu(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
            print("Features: ", extracted_features)
            features = np.vstack([features, extracted_features])
            labels = np.append(labels, label)
        print("Extracted features from %s, done" % (sub_dir))
    return features, labels

In [ ]:
features, labels = parse_audio_files(PATH, audio_subdirectories) #(parent dir,sub dirs)
np.save('feat.npy', features)
np.save('label.npy', labels)

<ipython-input-30-da1dc1e3a2f1>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  labels = np.empty(0, dtype=np.int)


Actual File Name:  /content/drive/MyDrive/Proyecto ML/audio_data/001 - Low/Interview with a Filippines Woman segment 22 - W.mp3
Features:  [3.97350993 3.26086957 3.97350993 1.         0.66       0.
 0.         1.         0.66       0.         1.5        0.82065217]
Actual File Name:  /content/drive/MyDrive/Proyecto ML/audio_data/001 - Low/Avalinguo - Victor and Abraham segment 131.mp3
Features:  [6.52173913 6.52173913 6.52173913 0.         0.         0.
 0.         0.         0.         0.         1.5        1.        ]
Actual File Name:  /content/drive/MyDrive/Proyecto ML/audio_data/001 - Low/Avalinguo - Victor and Abraham segment 119.mp3
Features:  [3.57142857 3.57142857 3.57142857 0.         0.         0.
 0.         0.         0.         0.         1.15384615 1.        ]
Actual File Name:  /content/drive/MyDrive/Proyecto ML/audio_data/001 - Low/Avalinguo - Dana and Konay segment 4 - D.mp3
Features:  [2.80373832 1.80722892 2.80373832 1.         1.18       0.
 0.         1.         1

KeyboardInterrupt: ignored

# Keep alive

In [ ]:
import time
while True:
    print("Keep alive")
    time.sleep(30)

Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive
Keep alive


KeyboardInterrupt: ignored